### Importações de bibliotecas

In [4]:
# Importacoes de bibliotecas

import pandas as pd # Analise de dados
from datetime import datetime, timedelta # Lidar om datas (diferencas, conversoes, etc.)
from dateutil.relativedelta import relativedelta # Tambem de lidar com datas
import statsmodels.api as sm # Biblioteca de onde vem a funcao para regressao (OLS)
from itertools import combinations # Funcao para iterar entre possiveis listas (para otimizar o AIC)
from random import randint

# Funcoes para aprendizado de maquina, ainda nao foi tao util
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from scipy import stats # Biblioteca para algumas metricas estatisticas, como trimmed mean
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [5]:
###################################################################################
######################### ESCOLHA DE VARIAVEIS RELEVANTES #########################
###################################################################################

df_dict = {'sp': [randint(0, 1) for _ in range(100)],
           'meses_cliente': [randint(1, 15) for _ in range(100)],
           'idoso': [randint(0, 1) for _ in range(100)],
           'brinde': [randint(0, 1) for _ in range(100)],
           'churn': [randint(0, 1) for _ in range(100)]}

df = pd.DataFrame(df_dict)

### Regressao simples
# Suponha que você tenha uma lista de variáveis explicativas
variaveis_explicativas_list = ['sp', 'meses_cliente', 'idoso', 'brinde']

# Inicialize uma lista para armazenar todas as combinações
todas_combinacoes = []
todas_combinacoes_list = []

# Gere todas as combinações possíveis
for r in range(1, len(variaveis_explicativas_list) + 1):
    combinacoes = combinations(variaveis_explicativas_list, r)
    todas_combinacoes.extend(combinacoes)

# Agora, todas_combinacoes contém todas as combinações possíveis
for combinacao in todas_combinacoes:
    todas_combinacoes_list.append(list(combinacao))

#######################################################################




###
# Suponha que você tenha um DataFrame df com seus dados
X = df[variaveis_explicativas_list]  # Variáveis explicativas
y = df['churn']  # Variável resposta

# Inicialize variáveis para armazenar o melhor modelo e seu AIC
melhor_modelo = None
melhor_aic = float('inf')

# Lista de combinacoes possiveis de variaveis
lista_de_combinacoes_de_variaveis = todas_combinacoes_list

# Loop sobre os modelos candidatos
for variaveis_explicativas in lista_de_combinacoes_de_variaveis:
    modelo = sm.OLS(y, sm.add_constant(X[variaveis_explicativas])).fit()  # Ajustar o modelo
    aic = modelo.aic  # Calcular o AIC

    # Verificar se o AIC é menor do que o melhor AIC atual
    if aic < melhor_aic:
        melhor_aic = aic
        melhor_modelo = modelo

# O melhor modelo e seu AIC estarão armazenados em melhor_modelo e melhor_aic
print("Melhor AIC:", melhor_aic)
print("Melhor Modelo:", melhor_modelo.summary())






# Modelo com as variaveis que me vieram a telha (para teste de predição)

modelo_use = sm.OLS(y, sm.add_constant(X[['sp', 'brinde']])).fit()
parametros = modelo_use.params
entrada = {'sp': 1, 'meses_cliente': 0, 'brinde': 0}
churn = parametros.const + parametros.sp*entrada['sp']  + parametros.brinde*entrada['brinde']








###### TESTE - APRENDIZADO DE MAQUINA ######

# Divida os dados em conjuntos de treinamento e teste
X = df[['sp', 'meses_cliente', 'idoso', 'brinde']]
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crie e treine o modelo de Regressão Logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avalie o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Acurácia:", accuracy)
print("Matriz de Confusão:")
print(confusion)
print("Relatório de Classificação:")
print(report)


# Suponha que você já tenha treinado o modelo e armazenado em 'model'

def prever_churn(model, sp, meses_cliente, idoso, brinde):
    # Crie um DataFrame com os parâmetros fornecidos
    dados = pd.DataFrame({'sp': [sp], 'meses_cliente': [meses_cliente], 'idoso': [idoso], 'brinde': [brinde]})
    
    # Faça a previsão usando o modelo
    previsao = model.predict(dados)
    
    # Retorne a previsão (0 ou 1)
    return previsao[0]

# Exemplo de uso da função
churn_previsto = prever_churn(model, sp=0, meses_cliente=0, idoso=1, brinde=1)

print("Previsão de Churn:", churn_previsto)

Melhor AIC: 136.55658647059678
Melhor Modelo:                             OLS Regression Results                            
Dep. Variable:                  churn   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     8.715
Date:                Wed, 20 Dec 2023   Prob (F-statistic):            0.00395
Time:                        09:48:57   Log-Likelihood:                -66.278
No. Observations:                 100   AIC:                             136.6
Df Residuals:                      98   BIC:                             141.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 